## Environment setup

Prepare `pytorch-1.5.1`, `cuda-10.1`, and `cudnn-v7.6.5`

In [ ]:
!nvidia-smi

Tue Mar 15 08:35:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 5000     Off  | 00000000:00:05.0 Off |                  Off |
| 33%   27C    P8     8W / 230W |      0MiB / 16125MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone --recursive git@github.com:ScrapPaper/off-belief-small.git

Cloning into 'off-belief-small'...
remote: Enumerating objects: 226, done.
remote: Counting objects: 100% (226/226), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 226 (delta 64), reused 221 (delta 59), pack-reused 0
Receiving objects: 100% (226/226), 4.26 MiB | 7.97 MiB/s, done.
Resolving deltas: 100% (64/64), done.
Submodule 'third_party/pybind11' (https://github.com/pybind/pybind11.git) registered for path 'third_party/pybind11'
Cloning into '/notebooks/off-belief-small/third_party/pybind11'...
remote: Enumerating objects: 23417, done.        
remote: Counting objects: 100% (1292/1292), done.        
remote: Compressing objects: 100% (505/505), done.        
remote: Total 23417 (delta 809), reused 1051 (delta 716), pack-reused 22125        
Receiving objects: 100% (23417/23417), 8.54 MiB | 11.41 MiB/s, done.
Resolving deltas: 100% (16229/16229), done.
Submodule path 'third_party/pybind11': checked out 'a1b71df137e015d44f7e31f7b6d4807253fb7871'
Submodule 'tool

In [ ]:
!nvcc --version # cuda-10.1
!cat /usr/include/cudnn.h | grep CUDNN_MAJOR -A 2 # cudnn-v7.6.5
!ls -al /usr/local/cuda-10.0/compat/libcuda.so.410.104

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
#define CUDNN_MAJOR 7
#define CUDNN_MINOR 6
#define CUDNN_PATCHLEVEL 2
--
#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

#include "driver_types.h"
-rw-r--r-- 1 root root 15672664 Feb  6  2019 /usr/local/cuda-10.0/compat/libcuda.so.410.104


In [ ]:
!ls -al /usr/lib/x86_64-linux-gnu/libcuda.so.*

lrwxrwxrwx 1 root root       20 Mar 15 08:21 /usr/lib/x86_64-linux-gnu/libcuda.so.1 -> libcuda.so.460.91.03
-rw-r--r-- 1 root root 15672664 Feb  6  2019 /usr/lib/x86_64-linux-gnu/libcuda.so.410.104
-rw-r--r-- 1 root root        0 Nov 24  2020 /usr/lib/x86_64-linux-gnu/libcuda.so.450.36.06
-rw-r--r-- 1 root root 21795104 Jul  2  2021 /usr/lib/x86_64-linux-gnu/libcuda.so.460.91.03


In [1]:
!apt update
!apt install -y build-essential libpq-dev libssl-dev openssl libffi-dev
!apt install -y python3-pip python3.7-dev
!apt install python3.7
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.6 1
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 2
!update-alternatives --set python3 /usr/bin/python3.7
!ln -sf /usr/bin/python3 /usr/local/bin/python
!ln -sf /usr/bin/python3 /usr/local/bin/python3
!python -m pip install --upgrade pip
!python -m pip install cython
!python -m pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html --progress-bar off
!python -m pip install psutil cmake==3.15.3

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]               
Get:3 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [909 kB]
Get:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [21.1 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1498 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2734 kB]33m
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 Packages [45.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]   
Get:11 http://archive.ubuntu.com/ubuntu bionic/main amd64 Packages [1344 kB]   
Get:12 http://archive.ubuntu.com/ubuntu bionic/restricted amd64 Packages

In [2]:
# avoid tensor operation using all cpu cores
%env OMP_NUM_THREADS 1

env: OMP_NUM_THREADS=1


## Compile learning environment

In [3]:
%cd /notebooks/off-belief-small/
!rm -rf build
!mkdir build
%cd build
!cmake ..
!make -j10

/notebooks/off-belief-small
/notebooks/off-belief-small/build
-- The C compiler identification is GNU 7.4.0
-- The CXX compiler identification is GNU 7.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Traceback (most recent call last):
  File "/notebooks/off-belief-small/get_pybind_flags.py", line 5, in <module>
    val = getattr(torch._C, f"_PYBIND11_{name}")
AttributeError: module 'torch._C' has no attribute '_PYBIND11_COMPILER_TYPE'
-- Found PythonInterp: /usr/bin/python3.7 (found suitable version "3.7.13", minimum required is "3.7") 
-

## Perform reinforcement learning

* **iql.sh**
```c
num_thread:          80   -> 8 // (quad-core CPU)
batchsize:           128  -> 64
act_device:          c:1  -> c:0 // (only 1 GPU)
belief_device:       c:1  -> c:0
rnn_hid_dim:         512  -> 256 // (reduced complexity for Hanabi-Small)
```

* **belief_obl0.sh**
```c
num_thread:          80   -> 8 // (quad-core CPU)
num_game_per_thread: 80   -> 40 // (reduce CPU load)
batchsize:           128  -> 64
hid_dim:             512  -> 256 // (reduced complexity for Hanabi-Small)
act_device:          c:1  -> c:0 // (only 1 GPU)
```

In [5]:
%cd ../pyhanabi

/notebooks/off-belief-small/pyhanabi


In [17]:
# iql
!cp -n scripts/iql.sh scripts/iql.sh.old
!sed -r -e "s/(--save_dir)[^\\]*/\\1 \\/notebooks\\/models /g" \\
        -e "s/(--num_thread)[^\\]*/\\1 8 /g" \\
        -e "s/(--batchsize)[^\\]*/\\1 64 /g" \\
        -e "s/(--act_device)[^\\]*/\\1 cuda:0 /g" \\
        scripts/iql.sh.old > scripts/iql.sh
!echo "       --belief_device cuda:0 \\" >> scripts/iql.sh
!echo "       --rnn_hid_dim 256 \\" >> scripts/iql.sh
!cat scripts/iql.sh

# Copyright (c) Facebook, Inc. and its affiliates.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#
#!/bin/bash
python selfplay.py \
       --save_dir /notebooks/models \
       --num_thread 8 \
       --num_game_per_thread 80 \
       --method iql \
       --sad 0 \
       --lr 6.25e-05 \
       --eps 1.5e-05 \
       --gamma 0.999 \
       --seed 2254257 \
       --burn_in_frames 10000 \
       --replay_buffer_size 100000 \
       --batchsize 64 \
       --epoch_len 1000 \
       --num_epoch 2000 \
       --num_player 2 \
       --net lstm \
       --num_lstm_layer 2 \
       --multi_step 3 \
       --train_device cuda:0 \
       --act_device cuda:0 \
       --belief_device cuda:0 \
       --rnn_hid_dim 256 \


In [18]:
# belief
!cp -n scripts/belief_obl0.sh scripts/belief_obl0.sh.old
!sed -r -e "s/(--save_dir)[^\\]*/\\1 \\/notebooks\\/belief_obl0 /g" \\
        -e "s/(--num_thread)[^\\]*/\\1 8 /g" \\
        -e "s/(--num_game_per_thread)[^\\]*/\\1 40 /g" \\
        -e "s/(--batchsize)[^\\]*/\\1 64 /g" \\
        -e "s/(--hid_dim)[^\\]*/\\1 256 /g" \\
        -e "s/(--act_device)[^\\]*/\\1 cuda:0 /g" \\
        scripts/belief_obl0.sh.old > scripts/belief_obl0.sh
!cat scripts/belief_obl0.sh

# Copyright (c) Facebook, Inc. and its affiliates.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#
#!/bin/bash
python train_belief.py \
       --save_dir /notebooks/belief_obl0 \
       --num_thread 8 \
       --num_game_per_thread 40 \
       --batchsize 64 \
       --lr 6.25e-05 \
       --eps 1.5e-05 \
       --grad_clip 5 \
       --hid_dim 256 \
       --burn_in_frames 10000 \
       --replay_buffer_size 100000 \
       --epoch_len 1000 \
       --num_epoch 500 \
       --train_device cuda:0 \
       --act_device cuda:0 \
       --explore 1 \
       --policy exps/iql/model0.pthw \
       --seed 2254257 \
       --num_player 2 \
       --shuffle_color 0 \
       --rand 1 \


In [ ]:
!sh scripts/belief_obl0.sh

{'act_device': 'cuda:0',
 'batchsize': 64,
 'burn_in_frames': 10000,
 'clone_bot': 0,
 'dataset': '',
 'epoch_len': 1000,
 'eps': 1.5e-05,
 'explore': 1,
 'fc_only': 0,
 'grad_clip': 5.0,
 'hid_dim': 256,
 'inf_data_loop': 0,
 'load_model': 0,
 'lr': 6.25e-05,
 'max_len': 80,
 'num_epoch': 500,
 'num_game_per_thread': 40,
 'num_player': 2,
 'num_thread': 8,
 'policy': 'exps/iql/model0.pthw',
 'prefetch': 3,
 'rand': 1,
 'replay_buffer_size': 100000,
 'save_dir': '/notebooks/belief_obl0',
 'seed': 2254257,
 'shuffle_color': 0,
 'train_device': 'cuda:0'}
loading file from:  exps/iql/model0.pthw
feature_size: (191, 171, 151)
explore eps: [1]
avg explore eps: 1.0
no boltzmann act
ActGroup created
Finished creating 8 threads with 320 games and 8 actors
warming up replay buffer: 0
warming up replay buffer: 1374
warming up replay buffer: 3002
warming up replay buffer: 4571
warming up replay buffer: 6022
warming up replay buffer: 7477
warming up replay buffer: 8903
Success, Done
beginning of e

In [19]:
!cp -n scripts/obl1.sh scripts/obl1.sh.old
!sed -r -e "s/(--save_dir)[^\\]*/\\1 \\/notebooks\\/obl1 /g" \\
        -e "s/(--num_thread)[^\\]*/\\1 8 /g" \\
        -e "s/(--num_game_per_thread)[^\\]*/\\1 120 /g" \\
        -e "s/(--batchsize)[^\\]*/\\1 64 /g" \\
        -e "s/(--rnn_hid_dim)[^\\]*/\\1 256 /g" \\
        -e "s/(--act_device)[^\\]*/\\1 cuda:0 /g" \\
        -e "s/(--belief_device)[^\\]*/\\1 cuda:0 /g" \\
        scripts/obl1.sh.old > scripts/obl1.sh
!cat scripts/obl1.sh

# Copyright (c) Facebook, Inc. and its affiliates.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#
#!/bin/bash
python selfplay.py \
       --save_dir /notebooks/obl1 \
       --num_thread 8 \
       --num_game_per_thread 120 \
       --sad 0 \
       --act_base_eps 0.1 \
       --act_eps_alpha 7 \
       --lr 6.25e-05 \
       --eps 1.5e-05 \
       --grad_clip 5 \
       --gamma 0.999 \
       --seed 2254257 \
       --batchsize 64 \
       --burn_in_frames 10000 \
       --replay_buffer_size 100000 \
       --epoch_len 1000 \
       --num_epoch 1500 \
       --num_player 2 \
       --rnn_hid_dim 256 \
       --multi_step 1 \
       --train_device cuda:0 \
       --act_device cuda:0 \
       --num_lstm_layer 2 \
       --boltzmann_act 0 \
       --min_t 0.01 \
       --max_t 0.1 \
       --off_belief 1 \
       --num_fict_sample 10 \
       --belief_device cuda:0 \
       --belief_model 

In [ ]:
!sh scripts/obl1.sh

{'act_base_eps': 0.1,
 'act_device': 'cuda:0',
 'act_eps_alpha': 7.0,
 'actor_sync_freq': 10,
 'aux_weight': 0,
 'batchsize': 64,
 'belief_device': 'cuda:0',
 'belief_model': 'exps/belief_obl0/model0.pthw',
 'boltzmann_act': 0,
 'burn_in_frames': 10000,
 'clone_bot': '',
 'clone_t': 0.02,
 'clone_weight': 0.0,
 'epoch_len': 1000,
 'eps': 1.5e-05,
 'eta': 0.9,
 'eval_bomb': 0,
 'gamma': 0.999,
 'grad_clip': 5.0,
 'hide_action': 0,
 'load_model': 'None',
 'lr': 6.25e-05,
 'max_len': 80,
 'max_t': 0.1,
 'method': 'iql',
 'min_t': 0.01,
 'multi_step': 1,
 'net': 'publ-lstm',
 'num_epoch': 1500,
 'num_fict_sample': 10,
 'num_game_per_thread': 120,
 'num_lstm_layer': 2,
 'num_player': 2,
 'num_t': 80,
 'num_thread': 8,
 'num_update_between_sync': 2500,
 'off_belief': 1,
 'prefetch': 3,
 'priority_exponent': 0.9,
 'priority_weight': 0.6,
 'replay_buffer_size': 100000,
 'rnn_hid_dim': 256,
 'sad': 0,
 'save_dir': '/notebooks/obl1',
 'seed': 2254257,
 'shuffle_color': 0,
 'train_bomb': 0,
 'tra

EPOCH: 4
Speed: train: 1077.5, buffer_add: 502.0, buffer_size: 100039
Total Time: 0H 05M 02S, 302s
Total Sample: train: 320K, buffer: 173.129K
@@@Time
	sync and updating : 2 MS, 3.90%
	sample data       : 0 MS, 1.59%
	forward & backward: 38 MS, 65.11%
	update model      : 17 MS, 29.31%
	updating priority : 0 MS, 0.10%
@@@total time per iter: 59.33 ms
[4] Time spent = 59.40 s
4:boltzmann_t  [1000]: avg:   0.0000, min:   0.0000[   0], max:   0.0000[   0]
4:grad_norm    [1000]: avg:   0.0835, min:   0.0100[ 559], max:   0.3433[ 323]
4:loss         [1000]: avg:   0.0381, min:   0.0036[ 514], max:   0.1203[ 900]
4:rl_loss      [1000]: avg:   0.0723, min:   0.0427[ 509], max:   0.1158[ 390]
epoch 4, eval score: 0.3350, perfect: 0.00, model saved: True
epoch 4, success rate for sampling ficticious state: 99.87%
beginning of epoch:  5
available: 8.679 GB, used: 20.353 GB, free: 4.401 GB
EPOCH: 5
Speed: train: 1070.3, buffer_add: 529.8, buffer_size: 100026
Total Time: 0H 06M 02S, 362s
Total Sam

15:grad_norm   [1000]: avg:   0.3661, min:   0.0375[ 821], max:   1.0459[ 438]
15:loss        [1000]: avg:   0.1465, min:   0.0155[ 821], max:   0.2671[ 706]
15:rl_loss     [1000]: avg:   0.0603, min:   0.0322[ 482], max:   0.0993[ 184]
epoch 15, eval score: 0.9940, perfect: 0.00, model saved: False
epoch 15, success rate for sampling ficticious state: 99.89%
beginning of epoch:  16
available: 5.797 GB, used: 23.235 GB, free: 1.497 GB
EPOCH: 677
Speed: train: 1167.2, buffer_add: 285.8, buffer_size: 100006
Total Time: 10H 28M 14S, 37694s
Total Sample: train: 43.392M, buffer: 10.854M
@@@Time
	sync and updating : 2 MS, 3.92%
	sample data       : 1 MS, 1.94%
	forward & backward: 35 MS, 65.67%
	update model      : 15 MS, 28.36%
	updating priority : 0 MS, 0.11%
@@@total time per iter: 54.76 ms
[677] Time spent = 54.84 s
677:boltzmann_t[1000]: avg:   0.0000, min:   0.0000[   0], max:   0.0000[   0]
677:grad_norm  [1000]: avg:   2.1084, min:   0.6088[ 625], max:   6.5014[ 654]
677:loss       [

epoch 685, eval score: 6.8470, perfect: 0.10, model saved: False
epoch 685, success rate for sampling ficticious state: 99.84%
beginning of epoch:  686
available: 4.442 GB, used: 24.592 GB, free: 332.988 MB
EPOCH: 686
Speed: train: 1147.6, buffer_add: 286.8, buffer_size: 100032
Total Time: 10H 36M 35S, 38195s
Total Sample: train: 43.968M, buffer: 10.997M
@@@Time
	sync and updating : 2 MS, 3.61%
	sample data       : 1 MS, 1.98%
	forward & backward: 36 MS, 65.37%
	update model      : 16 MS, 28.95%
	updating priority : 0 MS, 0.09%
@@@total time per iter: 55.66 ms
[686] Time spent = 55.77 s
686:boltzmann_t[1000]: avg:   0.0000, min:   0.0000[   0], max:   0.0000[   0]
686:grad_norm  [1000]: avg:   2.1641, min:   0.3782[ 260], max:   5.4040[ 378]
686:loss       [1000]: avg:   0.6445, min:   0.2179[ 260], max:   0.9443[ 276]
686:rl_loss    [1000]: avg:   0.1165, min:   0.0606[ 904], max:   0.2816[ 816]
epoch 686, eval score: 6.7880, perfect: 0.00, model saved: False
epoch 686, success rate f

epoch 694, eval score: 6.7020, perfect: 0.10, model saved: False
epoch 694, success rate for sampling ficticious state: 99.81%
beginning of epoch:  695
available: 4.438 GB, used: 24.595 GB, free: 328.402 MB
EPOCH: 695
Speed: train: 1151.2, buffer_add: 288.3, buffer_size: 100002
Total Time: 10H 44M 54S, 38694s
Total Sample: train: 44.544M, buffer: 11.14M
@@@Time
	sync and updating : 2 MS, 4.27%
	sample data       : 0 MS, 1.73%
	forward & backward: 35 MS, 64.69%
	update model      : 16 MS, 29.21%
	updating priority : 0 MS, 0.09%
@@@total time per iter: 55.52 ms
[695] Time spent = 55.60 s
695:boltzmann_t[1000]: avg:   0.0000, min:   0.0000[   0], max:   0.0000[   0]
695:grad_norm  [1000]: avg:   2.1415, min:   0.8006[ 288], max:   5.5405[ 866]
695:loss       [1000]: avg:   0.6325, min:   0.2671[ 583], max:   0.9724[ 429]
695:rl_loss    [1000]: avg:   0.1132, min:   0.0541[ 860], max:   0.2549[ 494]
epoch 695, eval score: 6.8560, perfect: 0.00, model saved: False
epoch 695, success rate fo

...

epoch 1095, eval score: 7.0190, perfect: 0.10, model saved: False
epoch 1095, success rate for sampling ficticious state: 99.82%
beginning of epoch:  1096
available: 4.240 GB, used: 24.792 GB, free: 296.285 MB
EPOCH: 1096
Speed: train: 1089.1, buffer_add: 265.9, buffer_size: 100035
Total Time: 16H 56M 08S, 60968s
Total Sample: train: 70.208M, buffer: 17.516M
@@@Time
	sync and updating : 2 MS, 3.74%
	sample data       : 1 MS, 1.81%
	forward & backward: 38 MS, 65.14%
	update model      : 17 MS, 29.20%
	updating priority : 0 MS, 0.12%
@@@total time per iter: 58.68 ms
[1096] Time spent = 58.77 s
1096:boltzmann_t[1000]: avg:   0.0000, min:   0.0000[   0], max:   0.0000[   0]
1096:grad_norm [1000]: avg:   2.2276, min:   0.4182[ 345], max:   6.5032[ 236]
1096:loss      [1000]: avg:   0.6252, min:   0.1637[ 345], max:   0.9603[ 838]
1096:rl_loss   [1000]: avg:   0.1155, min:   0.0551[ 787], max:   0.2798[  66]
epoch 1096, eval score: 6.9510, perfect: 0.20, model saved: False
epoch 1096, succes

epoch 1104, eval score: 6.8960, perfect: 0.10, model saved: False
epoch 1104, success rate for sampling ficticious state: 99.81%
beginning of epoch:  1105
available: 4.279 GB, used: 24.754 GB, free: 497.117 MB
EPOCH: 1105
Speed: train: 1151.6, buffer_add: 284.6, buffer_size: 100012
Total Time: 17H 04M 36S, 61476s
Total Sample: train: 70.784M, buffer: 17.659M
@@@Time
	sync and updating : 2 MS, 3.63%
	sample data       : 0 MS, 1.79%
	forward & backward: 36 MS, 65.23%
	update model      : 16 MS, 29.26%
	updating priority : 0 MS, 0.09%
@@@total time per iter: 55.51 ms
[1105] Time spent = 55.58 s
1105:boltzmann_t[1000]: avg:   0.0000, min:   0.0000[   0], max:   0.0000[   0]
1105:grad_norm [1000]: avg:   2.3616, min:   0.6204[ 465], max:   6.7780[ 326]
1105:loss      [1000]: avg:   0.6496, min:   0.1987[  31], max:   0.9557[ 691]
1105:rl_loss   [1000]: avg:   0.1170, min:   0.0612[ 458], max:   0.3256[ 973]
epoch 1105, eval score: 6.8830, perfect: 0.10, model saved: False
epoch 1105, succes

epoch 1113, eval score: 6.8820, perfect: 0.20, model saved: False
epoch 1113, success rate for sampling ficticious state: 99.82%
beginning of epoch:  1114
available: 4.279 GB, used: 24.754 GB, free: 496.699 MB
EPOCH: 1114
Speed: train: 1145.4, buffer_add: 283.3, buffer_size: 100006
Total Time: 17H 12M 57S, 61977s
Total Sample: train: 71.36M, buffer: 17.801M
@@@Time
	sync and updating : 2 MS, 4.00%
	sample data       : 1 MS, 1.83%
	forward & backward: 36 MS, 64.69%
	update model      : 16 MS, 29.37%
	updating priority : 0 MS, 0.11%
@@@total time per iter: 55.80 ms
[1114] Time spent = 55.88 s
1114:boltzmann_t[1000]: avg:   0.0000, min:   0.0000[   0], max:   0.0000[   0]
1114:grad_norm [1000]: avg:   2.2682, min:   0.7785[ 760], max:   7.5191[ 682]
1114:loss      [1000]: avg:   0.6382, min:   0.2829[ 189], max:   0.9640[  19]
1114:rl_loss   [1000]: avg:   0.1121, min:   0.0577[ 124], max:   0.3098[  74]
epoch 1114, eval score: 6.8440, perfect: 0.10, model saved: False
epoch 1114, success

epoch 1298, eval score: 7.0290, perfect: 0.10, model saved: False
epoch 1298, success rate for sampling ficticious state: 99.82%
beginning of epoch:  1299
available: 4.213 GB, used: 24.821 GB, free: 336.570 MB
EPOCH: 1299
Speed: train: 1138.0, buffer_add: 285.5, buffer_size: 100028
Total Time: 20H 04M 12S, 72252s
Total Sample: train: 83.2M, buffer: 20.733M
@@@Time
	sync and updating : 2 MS, 3.65%
	sample data       : 1 MS, 2.21%
	forward & backward: 36 MS, 64.64%
	update model      : 16 MS, 29.41%
	updating priority : 0 MS, 0.09%
@@@total time per iter: 56.18 ms
[1299] Time spent = 56.24 s
1299:boltzmann_t[1000]: avg:   0.0000, min:   0.0000[   0], max:   0.0000[   0]
1299:grad_norm [1000]: avg:   2.1836, min:   0.7767[  83], max:   6.4232[ 248]
1299:loss      [1000]: avg:   0.6148, min:   0.2242[ 344], max:   0.9320[ 178]
1299:rl_loss   [1000]: avg:   0.1135, min:   0.0520[ 952], max:   0.2839[ 189]
epoch 1299, eval score: 7.0210, perfect: 0.10, model saved: False
epoch 1299, success 

epoch 1307, eval score: 6.9170, perfect: 0.00, model saved: False
epoch 1307, success rate for sampling ficticious state: 99.82%
beginning of epoch:  1308
available: 4.209 GB, used: 24.825 GB, free: 326.238 MB
EPOCH: 1308
Speed: train: 1154.8, buffer_add: 277.7, buffer_size: 100005
Total Time: 20H 12M 36S, 72756s
Total Sample: train: 83.776M, buffer: 20.874M
@@@Time
	sync and updating : 2 MS, 3.79%
	sample data       : 0 MS, 1.78%
	forward & backward: 36 MS, 65.30%
	update model      : 16 MS, 29.01%
	updating priority : 0 MS, 0.13%
@@@total time per iter: 55.32 ms
[1308] Time spent = 55.42 s
1308:boltzmann_t[1000]: avg:   0.0000, min:   0.0000[   0], max:   0.0000[   0]
1308:grad_norm [1000]: avg:   2.2510, min:   0.4533[ 543], max:   8.2123[ 233]
1308:loss      [1000]: avg:   0.6147, min:   0.2302[ 543], max:   0.8916[ 720]
1308:rl_loss   [1000]: avg:   0.1099, min:   0.0434[ 204], max:   0.3012[ 198]
epoch 1308, eval score: 6.9370, perfect: 0.10, model saved: False
epoch 1308, succes